In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Load the data
data = pd.read_csv("../../data/F10.7.csv")  # Replace "your_data.csv" with the path to your data file

# Preprocess the data
X = data[['Julian day', 'Carringtonrotation', 'Observed Flux', 'Adjusted Flux', 'URSI Flux']].values
y = data['Adjusted Flux'].values  # Adjusted Flux will be our target variable

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape data for CNN input (assuming 1D convolution)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [3]:
# Build the CNN model
model = Sequential([
    #Add a 2d convolutional layer
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), padding='same'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    # MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)  
])

# Compile the model
model.compile(optimizer=Adam(), loss='mse')  # Using Mean Squared Error as loss function

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Predictions
y_pred = model.predict(X_test)

# Calculate additional metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)
print("Root Mean Squared Error (RMSE):", rmse)


Epoch 1/10
518/518 [==============================] - 1s 1ms/step - loss: 2062.7783 - val_loss: 938.6258
Epoch 2/10
518/518 [==============================] - 1s 1ms/step - loss: 1146.8008 - val_loss: 677.4485
Epoch 3/10
518/518 [==============================] - 1s 995us/step - loss: 506.5978 - val_loss: 5.4631
Epoch 4/10
518/518 [==============================] - 1s 979us/step - loss: 203.0256 - val_loss: 20.2494
Epoch 5/10
518/518 [==============================] - 0s 955us/step - loss: 194.6494 - val_loss: 26.0834
Epoch 6/10
518/518 [==============================] - 0s 963us/step - loss: 211.2842 - val_loss: 3.0085
Epoch 7/10
518/518 [==============================] - 1s 969us/step - loss: 203.4908 - val_loss: 3.8364
Epoch 8/10
518/518 [==============================] - 1s 965us/step - loss: 217.2457 - val_loss: 33.4829
Epoch 9/10
518/518 [==============================] - 1s 1ms/step - loss: 196.1759 - val_loss: 50.8094
Epoch 10/10
130/130 [==============================] - 0s 35

In [11]:
# Define the future data point
next_julian_day = 2453309.354
next_carrington_rotation = 2022.656
next_observed_flux = 137.5
next_adjusted_flux = 135.7
next_ursi_flux = 121.5

# Predict future values
future_data = np.array([[next_julian_day, next_carrington_rotation, next_observed_flux, next_adjusted_flux, next_ursi_flux]])
future_data_scaled = scaler.transform(future_data)
future_data_reshaped = future_data_scaled.reshape((future_data_scaled.shape[0], future_data_scaled.shape[1], 1))
future_prediction = model.predict(future_data_reshaped)
print("Predicted Adjusted Flux for Future Time:", future_prediction)

1/1 [==============================] - 0s 79ms/step
Predicted Adjusted Flux for Future Time: [[123.37208]]
